In [ ]:
#!pip install google-cloud-secret-manager
#!pip install numpy
#!pip install pandas
#!pip install cufflinks
#!pip install cx-Oracle

In [ ]:
import cx_Oracle
import datetime, json
import os
from google.cloud import secretmanager

import numpy as np
import pandas as pd
from pandas.tseries.offsets import Day

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv("UID"),
    password=os.getenv("PWD"),
    host = os.getenv("HOST"),
    service = os.getenv("SERVICE"),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data(sql):
    user = oracle_secrets['user'] + '[DVH_FAM_PP]' #f"[{schema}]"
    dsn_tns = cx_Oracle.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    try:
        conn = cx_Oracle.connect(user = user, password = oracle_secrets['password'], dsn = dsn_tns)
        df = pd.read_sql(sql = sql, con = conn)
        conn.close()
        return df
    except cx_Oracle.Error as error:
        print(error)

In [ ]:
%%time
# husk å legge til skjema navn dvh_fam_pp.

sql = """ 
SELECT A.PERIODE, DIM_TID.DATO, A.UTBET_FOM, A.UTBET_TOM, A.FK_PERSON1_MOTTAKER, A.FK_PERSON1_PLEIETRENGENDE, A.UTFALL, A.UTTAKSGRAD
,A.SAKSNUMMER, A.ARBEIDSFORHOLD_TYPE, A.PLEIEBEHOV
,B.GMT_OVERSE_ETABLERT_TILSYN_AARSAK, B.SOKERS_TAPTE_ARBEIDSTID, B.GMT_ANDRE_SOKERS_TILSYN, B.GMT_TILGJENGELIG_FOR_SOKER, B.GMT_ETABLERT_TILSYN, B.NATTEVAAK, B.BEREDSKAP
,AARSAK.AARSAK
FROM fam_pp_stonad A
JOIN fam_pp_perioder B ON A.FK_PP_PERIODER=B.PK_PP_PERIODER
LEFT OUTER JOIN fam_pp_periode_aarsak AARSAK ON AARSAK.FK_PP_PERIODER=B.PK_PP_PERIODER 
JOIN DT_P.DIM_TID DIM_TID ON DIM_TID.DIM_NIVAA=1 
AND DIM_TID.DAG_I_UKE<6 
AND DIM_TID.GYLDIG_FLAGG=1
AND A.YTELSE_TYPE='PSB'
AND A.UTBET_FOM<=DIM_TID.DATO 
AND A.UTBET_TOM>=DIM_TID.DATO
WHERE A.KILDESYSTEM='PP_VEDTAK' AND A.GYLDIG_FLAGG=0 AND A.PERIODE>202100 AND A.PERIODE<202206
and to_char(DIM_TID.DATO,'YYYYMM')=PERIODE"""

df = get_data(sql = sql)

In [ ]:
df.head(3)

# Bestilling av stønadsstatistikk i forbindelse med endringer i 9-11

- Trenger totalt antall saker per dag (som er i søknadsperioden).
- Hvilken tidsperiode uttrekket gjelder for
    - Fullt uttrekk for alle data fra K9 
    - Data per dag dataene gjelder for.

### Total antall saker per dag innvilget og ikke innvilget 

In [ ]:
antal_saker = df[['DATO', 'SAKSNUMMER', 'UTFALL']].pivot_table(index = 'DATO', columns = 'UTFALL',
                                                               values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()), margins = True)[:-1]

In [ ]:
antal_saker.columns = [''] * len(antal_saker.columns)
antal_saker.columns = ['ANT_SAKER_IKKE_OPP', 'ANT_SAKER_OPP', 'ANT_SAK_TOTALT'] 
antal_saker.head()

- I hvor mange saker er det opplyst om timer med tilsynsordning?
    - Antall saker med tilsynsordning per dag.

In [ ]:
filt = ((df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0)) 
tilsynsordning = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()))

In [ ]:
tilsynsordning.columns = [''] * len(tilsynsordning.columns)
tilsynsordning.columns =['Ant_SAK_MED_TILSYN']
tilsynsordning.head()

- I hvor mange av sakene med tilsynsordning er det vurdert nattevåk og/eller beredskap?
    - Antall saker med innvilget beredskap per dag.
    - Antall saker med avslått beredskap per dag.
    - Antall saker med innvilget nattevåk per dag.
    - Antall saker med avslått nattevåk per dag.
    - Antall saker med innvilget beredskap & nattevåk per dag.
    - Antall saker med avslått beredskap & nattevåk per dag.

In [ ]:
filt = ((df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
beredskap_df = df.loc[filt, ['DATO','SAKSNUMMER', 'BEREDSKAP']].pivot_table(index = 'DATO', columns = 'BEREDSKAP',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))
beredskap_df.columns = [''] * len(beredskap_df.columns)
beredskap_df.columns = ['BEREDSKAP_IKKE_OPP', 'BEREDSKAP_OPP']
beredskap_df.head()

In [ ]:
filt = ((df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
nattevaak_df = df.loc[filt, ['DATO','SAKSNUMMER', 'NATTEVAAK']].pivot_table(index = 'DATO', columns = 'NATTEVAAK',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))
nattevaak_df.columns = [''] * len(nattevaak_df.columns)
nattevaak_df.columns = ['NATTEVAAK_IKKE_OPP', 'NATTEVAAK_OPP']
nattevaak_df.head()

In [ ]:
filt = ((df['BEREDSKAP'] == 'OPPFYLT') & (df['NATTEVAAK'] == 'OPPFYLT') & (df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
nattevaak_og_beredskap_opp_df = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))

In [ ]:
nattevaak_og_beredskap_opp_df.columns = [''] * len(nattevaak_og_beredskap_opp_df.columns)
nattevaak_og_beredskap_opp_df.columns = ['BEREDSKAP_OG_NATTEVAAK_INNVILGET']
nattevaak_og_beredskap_opp_df.head()

In [ ]:
filt = ((df['BEREDSKAP'] == 'IKKE_OPPFYLT') & (df['NATTEVAAK'] == 'IKKE_OPPFYLT') & (df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
nattevaak_og_beredskap_ikke_opp_df = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))

In [ ]:
nattevaak_og_beredskap_ikke_opp_df.columns = [''] * len(nattevaak_og_beredskap_opp_df.columns)
nattevaak_og_beredskap_ikke_opp_df.columns = ['BEREDSKAP_OG_NATTEVAAK_IKKE_INNVILGET']
nattevaak_og_beredskap_ikke_opp_df.reindex(nattevaak_og_beredskap_opp_df.index).fillna(0).astype(int).tail(20)

In [ ]:
natt_bred_result = pd.concat([beredskap_df, nattevaak_df, nattevaak_og_beredskap_opp_df, nattevaak_og_beredskap_ikke_opp_df], 
                             axis=1).reindex(beredskap_df.index).fillna(value = 0)
natt_bred_result.astype(int).head()

- I hvor mange av sakene er det unntak fra gradering på grunn av nattevåk og/eller beredskap?
    - Antall saker der nattevåk og/eller beredskap blir brukt som unntak per dag.

In [ ]:
l = ['NATTEVÅK_OG_BEREDSKAP', 'BEREDSKAP', 'NATTEVÅK']
filt = ((df['GMT_ETABLERT_TILSYN'] > 0) & (df['UTFALL'] == 'OPPFYLT') & (df['GMT_OVERSE_ETABLERT_TILSYN_AARSAK'].isin(l)))
#filt = ( (df['UTFALL'] == 'OPPFYLT') & (((df['NATTEVAAK'] == 'OPPFYLT') | (df['BEREDSKAP'] == 'OPPFYLT')) | ((df['NATTEVAAK'] == 'OPPFYLT') & (df['BEREDSKAP'] == 'OPPFYLT')))) 
natt_bred_unntak = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(index='DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()) )

In [ ]:
natt_bred_unntak.columns = [''] * len(natt_bred_unntak.columns)
natt_bred_unntak.columns = ['ANT_SAK_NATT_OG_PRED_UNNTAK_FRA_GRAD']
natt_bred_unntak.head()

- Antall saker som graderes med årsak per dag:
    - GRADERT_MOT_TILSYN
    - AVKORTET_MOT_INNTEKT

In [ ]:
filt = ((df['UTFALL'] == 'OPPFYLT') & ((df['AARSAK'] == 'AVKORTET_MOT_INNTEKT') | (df['AARSAK'] == 'GRADERT_MOT_TILSYN')))
gradering_med_aarsak = df.loc[filt, ['DATO', 'AARSAK', 'SAKSNUMMER']].pivot_table(index = 'DATO', columns = 'AARSAK', values = 'AARSAK',
                                                           aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
gradering_med_aarsak.columns = [''] * len(gradering_med_aarsak.columns)
gradering_med_aarsak.columns =['ANT_SAK_AVKORTET_MOT_INNTEKT', 'ANT_SAK_GRADERT_MOT_TILSYN']
gradering_med_aarsak.head()

- Antall avslag med årsak per dag:
    - FOR_LAV_REST_PGA_ETABLERT_TILSYN 
    - FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE
    - FOR_LAV_TAPT_ARBEIDSTID
    - FOR_LAV_REST_PGA_ANDRE_SØKERE

In [ ]:
avslag_aarsak_list = ['FOR_LAV_REST_PGA_ETABLERT_TILSYN', 'FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE', 'FOR_LAV_TAPT_ARBEIDSTID', 'FOR_LAV_REST_PGA_ANDRE_SØKERE']
filt = df['AARSAK'].isin(avslag_aarsak_list)
avslag_aarsak = df.loc[filt, ['DATO', 'AARSAK', 'SAKSNUMMER']].pivot_table(index = 'DATO', columns = 'AARSAK', values = 'SAKSNUMMER',
                                                          aggfunc = lambda x: len(x.unique()), fill_value = 0)

In [ ]:
avslag_aarsak.columns = [''] * len(avslag_aarsak.columns)
avslag_aarsak.columns =['ANT_SAK_FOR_LAV_REST_PGA_ANDRE_SØKERE', 'ANT_SAK_FOR_LAV_REST_PGA_ETABLERT_TILSYN', 'ANT_SAK_FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE', 'ANT_SAK_FOR_LAV_TAPT_ARBEIDSTID']
avslag_aarsak.head()

- Hvor mange har ikke arbeid?
    - Alle aktiviteter bortsett fra selvstendig næringsdrivende, frilanser og/eller arbeidstaker.

In [ ]:
bortsett_liste = ['AT','SN','FL', None]
filt = (df['UTFALL'] == 'OPPFYLT') & (~df['ARBEIDSFORHOLD_TYPE'].isin(bortsett_liste))
ikke_arbeid = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique())
                                                ,fill_value = 0)

In [ ]:
ikke_arbeid.columns = [''] * len(ikke_arbeid.columns)
ikke_arbeid.columns =['ANT_SAK_UTEN_ARBEID']
ikke_arbeid.head()

- Hvor mange har ikke arbeid og oppgir tilsyn?

In [ ]:
arbeid_liste = ['AT','SN','FL']
filt = ((df['GMT_ETABLERT_TILSYN'] > 0) & (df['UTFALL'] == 'OPPFYLT') & (~df['ARBEIDSFORHOLD_TYPE'].isin(arbeid_liste)))
saker_arbeid_tilsyn = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER',
                                                aggfunc = lambda x: len(x.unique()), fill_value = 0)

In [ ]:
saker_arbeid_tilsyn.columns = [''] * len(saker_arbeid_tilsyn.columns)
saker_arbeid_tilsyn.columns =['ANT_SAK_IKKE_ARBEID_OG_OPPGIR_TILSYN']
saker_arbeid_tilsyn.head()

- Antall saker med arbeid og oppgitt tilsyn per dag.

In [ ]:
arbeid_liste = ['AT','SN','FL']
filt = ((df['GMT_ETABLERT_TILSYN'] > 0) & (df['UTFALL'] == 'OPPFYLT') & (df['ARBEIDSFORHOLD_TYPE'].isin(arbeid_liste)))
saker_har_arbeid_tilsyn = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER',
                                                aggfunc = lambda x: len(x.unique()), fill_value = 0)

In [ ]:
saker_har_arbeid_tilsyn.columns = [''] * len(saker_har_arbeid_tilsyn.columns)
saker_har_arbeid_tilsyn.columns =['ANT_SAK_ER I_ARBEID_OG_OPPGIR_TILSYN']
saker_har_arbeid_tilsyn.head()

## Gradering mot annen omsorgsperson:

- Hvor mange saker har flere søkere i samme periode?
    - Antall saker for et gitt antall søkere (1,2,3,4 eller 5) per dag. (ikke så viktig)

In [ ]:
filt = (df['UTFALL'] == 'OPPFYLT')
d = df.loc[filt,['PERIODE', 'FK_PERSON1_MOTTAKER', 'FK_PERSON1_PLEIETRENGENDE']].drop_duplicates(keep='first').groupby(['PERIODE', 'FK_PERSON1_PLEIETRENGENDE'])['FK_PERSON1_MOTTAKER'].count().reset_index()

In [ ]:
#filt = ((d['PERIODE'] == '202201') & (d['FK_PERSON1_MOTTAKER'] == 2) | (d['FK_PERSON1_MOTTAKER'] == 3))
ant_mottakere_per_sak = pd.crosstab(d['PERIODE'], d['FK_PERSON1_MOTTAKER'])

In [ ]:
ant_mottakere_per_sak.columns = [''] * len(ant_mottakere_per_sak.columns)
ant_mottakere_per_sak.columns = ['ANT_SAK_EN_MOTTAKER', 'ANT_SAK_TO_MOTTAKER','ANT_SAK_TRE_MOTTAKER']
ant_mottakere_per_sak.head()

In [ ]:
filt = (df['UTFALL'] == 'OPPFYLT')
d_ = df.loc[filt,['DATO', 'FK_PERSON1_MOTTAKER', 'FK_PERSON1_PLEIETRENGENDE']].drop_duplicates(keep='first').groupby(['DATO', 'FK_PERSON1_PLEIETRENGENDE'])['FK_PERSON1_MOTTAKER'].count().reset_index()

In [ ]:
ant_mottakere_per_sak_ = pd.crosstab(d_['DATO'], d_['FK_PERSON1_MOTTAKER'])

In [ ]:
ant_mottakere_per_sak_.columns = [''] * len(ant_mottakere_per_sak_.columns)
ant_mottakere_per_sak_.columns = ['ANT_SAK_EN_MOTTAKER', 'ANT_SAK_TO_MOTTAKER','ANT_SAK_TRE_MOTTAKER']
ant_mottakere_per_sak_.head()

- I hvor mange saker er det kun behov for 1 omsorgsperson?
    - Antall saker for en gitt pleiegrad (0%, 100%, eller 200%) per dag

In [ ]:
filt = (df['UTFALL'] == 'OPPFYLT')
pleiebehov = df.loc[filt,['DATO', 'PLEIEBEHOV', 'SAKSNUMMER']].pivot_table(index = 'DATO',columns = 'PLEIEBEHOV', values = 'SAKSNUMMER',
                                                    aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
pleiebehov.columns = [''] * len(pleiebehov.columns)
pleiebehov.columns =['ANT_SAK_100_PLEIEBEHOV', 'ANT_SAK_200_PLEIEBEHOV']
pleiebehov.head()

## Gradering mot arbeid:

- Dette uttrekket er ikke per dag, men totalt over alle periodene til en fagsak:
    - Hvor mange saker graderes/avslås uten årsakene AVKORTET_MOT_INNTEKT og/eller FOR_LAV_TAPT_ARBEIDSTID
    - Hvor mange saker graderes/avslås uten årsaken GRADERT_MOT_TILSYN

### Antall saker graderes/avslås uten årsakene AVKORTET_MOT_INNTEKT og/eller FOR_LAV_TAPT_ARBEIDSTID

In [ ]:
# NY
# Hvor mange graders grunn tilsyn (kreteriet fra møtet)
filt = (df['AARSAK'] == 'GRADERT_MOT_TILSYN')
df.loc[filt][['UTFALL', 'SAKSNUMMER']].pivot_table(index = 'UTFALL', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()))

In [ ]:
bortsett_aarsak = ['AVKORTET_MOT_INNTEKT', 'FOR_LAV_TAPT_ARBEIDSTID']
filt = (~df['AARSAK'].isin(bortsett_aarsak)) & (df['UTTAKSGRAD'] < 100)
df.loc[filt][['UTFALL', 'SAKSNUMMER']].pivot_table(index = 'UTFALL', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()))

### Antall saker graderes/avslås uten årsaken GRADERT_MOT_TILSYN

In [ ]:
# NY
# Hvor mange graders grunn inntekt (kreteriet fra møtet)
filt = (df['AARSAK'] == 'AVKORTET_MOT_INNTEKT') 
df.loc[filt][['UTFALL', 'SAKSNUMMER']].pivot_table(index = 'UTFALL', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()))

In [ ]:
filt = (df['AARSAK'] != 'GRADERT_MOT_TILSYN') & (df['UTTAKSGRAD'] < 100) 
df.loc[filt,['UTFALL', 'SAKSNUMMER']].pivot_table(index = 'UTFALL', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))

## §8-13 og gradering mot 20 %

 - Antall saker med årsak FOR_LAV_TAPT_ARBEIDSTID per dag

In [ ]:
filt = (df['AARSAK'] == 'FOR_LAV_TAPT_ARBEIDSTID')
sak_lav_tapt_arbeidstid = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()), fill_value = 0)
sak_lav_tapt_arbeidstid.columns = [''] * len(sak_lav_tapt_arbeidstid.columns)
sak_lav_tapt_arbeidstid.columns = ['ANT_SAK_MED_FOR_LAV_TAPT_ARBEIDSTID']
sak_lav_tapt_arbeidstid.head()

 - Antallet saker som ikke har minst én søknadsperiode som er lenger enn 5 dager sammenhengende.

In [ ]:
sql_ = """ 
SELECT A.PERIODE, A.UTBET_FOM, A.UTBET_TOM, A.UTFALL, A.SAKSNUMMER
FROM fam_pp_stonad A
WHERE A.KILDESYSTEM='PP_VEDTAK' AND A.GYLDIG_FLAGG=0 AND A.PERIODE>202100 AND A.PERIODE<202206
"""
df_ = get_data(sql = sql_)

In [ ]:
df_['ANTALL_DAGER'] = df_['UTBET_TOM'].sub(df_['UTBET_FOM']).mod(365*Day()).dt.days + 1

In [ ]:
filt = ((df_['ANTALL_DAGER'] == 5))
saker_med_søknadsperiode_5 = df_.loc[filt, 'SAKSNUMMER'].unique() # liste med alle saker som har én søknadsperiode på 5 dager i alle periodene januar, februer, mars, april

In [ ]:
filt = (~(df_['SAKSNUMMER'].isin(saker_med_søknadsperiode_5)))
df_.loc[filt].head()

In [ ]:
df_['WEEK_UTBET_FOM'] = df_['UTBET_FOM'].dt.week

In [ ]:
filt = (~(df_['SAKSNUMMER'].isin(saker_med_søknadsperiode_5)))
saker_med_ant_dager_sum = df_.loc[filt].drop_duplicates(keep='first').groupby(['WEEK_UTBET_FOM', 'SAKSNUMMER'])['ANTALL_DAGER'].sum().reset_index()

In [ ]:
# filtrer vekk sakker som har 5 dager etter summeringa, da får vi kun saker som ikke oppfyler det 5 dager kravet
filt = (saker_med_ant_dager_sum['ANTALL_DAGER'] == 5)
saker_med_søknadsperiode_5_etter_sum = saker_med_ant_dager_sum.loc[filt, 'SAKSNUMMER'].unique()

filt = (~(saker_med_ant_dager_sum['SAKSNUMMER'].isin(saker_med_søknadsperiode_5_etter_sum)))
len(saker_med_ant_dager_sum.loc[filt,'SAKSNUMMER'].unique())

In [ ]:
#n.to_excel('saker_oppfyller_ikke_krav.xlsx')

### Antall nye saker per periode

In [ ]:
nye_saker = df[['PERIODE', 'SAKSNUMMER']].drop_duplicates(keep='first').sort_values(by = 'PERIODE')
nye_saker['PERIODE'] = pd.to_datetime(nye_saker['PERIODE'],format='%Y%m') 

In [ ]:
d1 = nye_saker.groupby('PERIODE')['SAKSNUMMER'].value_counts()
d2 = d1.unstack().diff().replace({0: False, np.nan: True}).stack()
nye_saker['NY_SAK'] = nye_saker.set_index(['PERIODE', 'SAKSNUMMER']).index.map(d2)

In [ ]:
nye_saker.head()

In [ ]:
filt = (nye_saker['NY_SAK'] == True)
nye_saker.loc[filt].head()

In [ ]:
ant_ny_sak_per_periode = pd.crosstab(nye_saker.loc[filt,'PERIODE'], nye_saker.loc[filt,'NY_SAK'])

In [ ]:
ant_ny_sak_per_periode.head()

In [ ]:
ant_ny_sak_per_periode.columns = [''] * len(ant_ny_sak_per_periode.columns)
ant_ny_sak_per_periode.columns = ['ANT_NYE_SAKER']
ant_ny_sak_per_periode.head()

In [ ]:
ant_ny_sak_per_periode = pd.concat([ant_ny_sak_per_periode, nye_saker.groupby('PERIODE')['SAKSNUMMER'].nunique()], axis=1)

In [ ]:
ant_ny_sak_per_periode.columns = [''] * len(ant_ny_sak_per_periode.columns)
ant_ny_sak_per_periode.columns = ['ANT_NYE_SAKER', 'TOTAL_ANT_SAKER']
ant_ny_sak_per_periode = ant_ny_sak_per_periode[['TOTAL_ANT_SAKER', 'ANT_NYE_SAKER']] # endrer rekkefølge på kolonene 

In [ ]:
ant_ny_sak_per_periode['NYE_SAKER_PERCENT'] = round((ant_ny_sak_per_periode['ANT_NYE_SAKER'] / ant_ny_sak_per_periode['TOTAL_ANT_SAKER']) * 100, 2)
ant_ny_sak_per_periode.head()

In [ ]:
ant_ny_sak_per_periode[['TOTAL_ANT_SAKER', 'ANT_NYE_SAKER']].iplot()

### Resultat tabell

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
slutt_df = pd.concat([antal_saker,tilsynsordning,natt_bred_result,natt_bred_unntak,gradering_med_aarsak,avslag_aarsak,ikke_arbeid,saker_arbeid_tilsyn, saker_har_arbeid_tilsyn, pleiebehov, sak_lav_tapt_arbeidstid], 
                             axis=1).reindex(antal_saker.index).fillna(value = 0)

In [ ]:
slutt_df.astype(int)

In [ ]:
filt = (df['DATO'] == '2022-05-09')
m = df.loc[filt]

In [ ]:
z = df.loc[filt][['FK_PERSON1_MOTTAKER', 'ARBEIDSFORHOLD_TYPE']].pivot_table(index='FK_PERSON1_MOTTAKER', values = 'ARBEIDSFORHOLD_TYPE'
                                                                         , aggfunc = lambda x: len(x.unique())).reset_index()

In [ ]:
filt = (z['ARBEIDSFORHOLD_TYPE'] > 1)
p = z.loc[filt]

In [ ]:
l = np.array(p['FK_PERSON1_MOTTAKER'])
filt = (m['FK_PERSON1_MOTTAKER'].isin(l))
m.loc[filt].sort_values(by = 'FK_PERSON1_MOTTAKER')[:50]

In [ ]:
filt = (df['DATO'] == '2022-05-09')
n = df.loc[filt, ['DATO', 'FK_PERSON1_MOTTAKER', 'FK_PERSON1_PLEIETRENGENDE']].drop_duplicates(keep='first').groupby(['DATO', 'FK_PERSON1_MOTTAKER'])['FK_PERSON1_PLEIETRENGENDE'].count().reset_index()

In [ ]:
filt = (n['FK_PERSON1_PLEIETRENGENDE'] > 1)
n.loc[filt]

In [ ]:
df['ARBEIDSFORHOLD_TYPE'].unique()

In [ ]:
filt = (df['FK_PERSON1_MOTTAKER'] == 1553508705) & (df['DATO'] == '2022-05-09')
df.loc[filt]

In [ ]:
filt = (df['GMT_ETABLERT_TILSYN'] > 0) & (df['UTFALL'] == 'OPPFYLT') & (df['DATO'] == '2022-05-09')
m = df.loc[filt][['FK_PERSON1_MOTTAKER', 'ARBEIDSFORHOLD_TYPE']].pivot_table(index='FK_PERSON1_MOTTAKER', values = 'ARBEIDSFORHOLD_TYPE'
                                                                         , aggfunc = lambda x: len(x.unique())).reset_index()

In [ ]:
filt = (m['ARBEIDSFORHOLD_TYPE'] > 1)
p = m.loc[filt]

In [ ]:
l = np.array(p['FK_PERSON1_MOTTAKER'])
filt = (m['FK_PERSON1_MOTTAKER'].isin(l))
m.loc[filt].sort_values(by = 'FK_PERSON1_MOTTAKER')[:50]

In [ ]:
df['AARSAK'].unique()

In [ ]:
filt = (df['UTFALL'] == 'OPFFYLT') & (df['AARSAK'] == 'FULL_DEKNING')
df.loc[filt]

In [ ]:
filt =  (df['FK_PERSON1_PLEIETRENGENDE'] == 1348914165) & (df['SOKERS_TAPTE_ARBEIDSTID'] == 100)
df.loc[filt]